# 03: Velocity inversion

# NOTE: still need to organize this code more

In [ ]:
#| default_exp invert

In [ ]:
#| hide
from nbdev.showdoc import *
import nbdev 
from nbdev import nbdev_export

In [ ]:
#| export
import numpy as np
import pyproj
import matplotlib.path as path
import s3fs
import zarr
import matplotlib.pyplot as plt
import scipy
from datetime import timedelta
from tqdm import tqdm
import xarray as xr
import re
import pandas as pd
import geopandas as gpd
import matplotlib.path as mplp
import ipyleaflet as ipyl
from ipyleaflet import WMSLayer
import ipywidgets as ipyw
import json
import pandas as pd
from ipyleaflet import Map, WMSLayer, basemaps
from ipywidgets import HTML
from owslib.wms import WebMapService

In [ ]:
from ITS_LIVE_Analysis import setup, interactive

In [ ]:
#| export
urls = []

In [ ]:
map = interactive.Widget()

In [ ]:
map.display()

In [ ]:
coords, gpdf, urls = interactive.return_clicked_info(map)

In [ ]:
glacier1 = {'RGIID': gpdf.iloc[0]['RGIID'], 
           'coords': coords[0],
            'url':urls[0],
            'gdf':gpdf
           }

In [ ]:
coords_utm_gdf = setup.point_to_gdf(glacier1['coords']).to_crs('EPSG:32643')
coords_utm_gdf

coords_utm_ls = [coords_utm_gdf['y'].iloc[0], coords_utm_gdf['x'].iloc[0]]
coords_utm_ls

urls = [glacier1['url']]

gdf_list = [glacier1['gdf']]

In [ ]:
u

['http://its-live-data.s3.amazonaws.com/datacubes/v2/N30E070/ITS_LIVE_vel_EPSG32643_G0120_X650000_Y3950000.zarr']

In [ ]:
#| export
# Gather the points on the boundary of the glacier (should later be implemented as a GUI loading the points of a glacier's periphery)
#boundary_points = pickle.load(open('boundary.p','rb'))

# Modify the urls so they can be opened by zarr (replace 'http' by 's3' and delete '.s3.amazonaws.com')
#urls = [url.replace('http','s3') for url in urls]
try: 
    urls = [re.sub(r'http', 's3', url) for url in urls]
    urls = [re.sub(r'\.s3\.amazonaws\.com', '', url) for url in urls]
    # Create storing arrays for the coordinates on-glacier
    X_valid = []
    Y_valid = []
    X_tot = []
    Y_tot = []
    
    # Create an empty directoryimport pickle to hold many variables all tied to the datacubes
    data_dict = {}
    
    # We iterate through the different datacubes so they can each have one instance of the variables below
    for url in urls:
        zarr_store = None # To store the datacube's information and access its variables
        dates = None # To store the dates at which the inversion will give values
        A_m = None # 1st part of the design matrix
        reg_mat_Inv = None # Regularization in time, 2nd part of the design matrix
        mission = None # If you want to invert specifically for one mission in particular ('S1','L8','L9', etc...)
        index_sort = None # Indices representing the sorted dates (from older to most recent)
        inds_mission = None # Indices representing the sorted dates per mission chosen
        ind_tot = None # Indices representing the indices of the pixels on the GOI
        valid_idx = None # Easting and Northing values of the indices above
        proj_cube = None # Projection of the datacube
        
        # Create a dictionary entry for the URL with the desired subsets
        data_dict[url] = {
            'zarr_store': zarr_store,
            'dates': dates,
            'A_m': A_m,
            'reg_mat_Inv': reg_mat_Inv,
            'mission': mission,
            'index_sort': index_sort,
            'inds_mission': inds_mission,
            'dates': dates,
            'ind_tot': ind_tot,
            'valid_idx': valid_idx,
            'proj_cube': proj_cube
        }
except: 
    pass

## Datacubes Extent and Point Validity

Designed to grab the extents of the datacubes, and determines which pixel belongs to the GOI. This way we do not lose time inverting for empty pixels.

In [ ]:
#| export 

def get_extents(url, X_tot, Y_tot, X_valid, Y_valid, data_dict, mission, lamb, derivative, day_interval):

    # Open the zarr files
    fs = s3fs.S3FileSystem(anon=True)
    store = zarr.open(s3fs.S3Map(url, s3=fs))
   
    # Update the dictionnary
    data_dict[url]['zarr_store'] = store

    # Get the cube's projection
    proj_cube = store.attrs['projection']

    # Load X and Y of the dataset
    X = store['x'][:]
    Y = store['y'][:]

    # Store the arrays in the total list
    X_tot.append(X)
    Y_tot.append(Y)

    # Load dimensions
    shape_arr = store['v'].shape
    
    Xs, Ys = np.meshgrid(X, Y)
    points = np.array((Xs.flatten(), Ys.flatten())).T

    idx_valid = []
    
    for b in range(len(gdf_list)):
        mpath = mplp.Path(list(gdf_list[b]['geometry'].to_crs(proj_cube).boundary.explode(index_parts = True).iloc[0].coords))
        glacier_mask = mpath.contains_points(points).reshape(Xs.shape)
        # Grab the indices of the points inside the glacier
        idx_valid.append(np.array(np.where(glacier_mask==True)))
        
    idx_valid = np.hstack(idx_valid)
    # Store the valid indices
    data_dict[url]['valid_idx'] = idx_valid
    
    # Store the cube projection
    data_dict[url]['proj_cube'] = proj_cube
    
    # Store the coordinates of the valid Xs and Ys
    X_valid.append([Xs[idx_valid[0][i], idx_valid[1][i]] for i in range(len(idx_valid[0]))])
    Y_valid.append([Ys[idx_valid[0][i], idx_valid[1][i]] for i in range(len(idx_valid[0]))])
    
    return X_tot, Y_tot, X_valid, Y_valid

## Design Matrices

This function creates 1 design matrix per cube. Knowing that each cube has different time stamps (different image pairs), but the possible dates for the image pairs are the same for every pixel of the datacube, we can pre-compute 1 design-matrix for each cube. We will simply mask-out the rows that represent time steps for which our point being inverted does not have any value.

In [ ]:
#| export 
def design_matrices(url, mission, lamb, derivative, day_interval):

    # If you passed 'mission' as an argument, it grabs the appropriate values
    if mission:
        # Get the indices of the mission
        filt1 = np.where(data_dict[urls[url]]['zarr_store']['satellite_img1'][:] == mission)
        filt2 = np.where(data_dict[urls[url]]['zarr_store']['satellite_img2'][:] == mission)
        inds_mission = np.intersect1d(filt1[0],filt2[0])

        # Grab only the indices corresponding to the missions
        mid_dates = np.datetime64('1970-01-01') + np.array(data_dict[urls[url]]['zarr_store']['mid_date'][:], dtype='timedelta64[D]')[inds_mission]
        im1 = np.datetime64('1970-01-01') + np.array(data_dict[urls[url]]['zarr_store']['acquisition_date_img1'][:], dtype='timedelta64[D]')[inds_mission]
        im2 = np.datetime64('1970-01-01') + np.array(data_dict[urls[url]]['zarr_store']['acquisition_date_img2'][:], dtype='timedelta64[D]')[inds_mission]
    else:
        # If 'None' was passed as a mission argument, we grab all the available data.
        inds_mission = None
        mid_dates = np.datetime64('1970-01-01') + np.array(data_dict[urls[url]]['zarr_store']['mid_date'][:], dtype='timedelta64[D]')
        im1 = np.datetime64('1970-01-01') + np.array(data_dict[urls[url]]['zarr_store']['acquisition_date_img1'][:], dtype='timedelta64[D]')
        im2 = np.datetime64('1970-01-01') + np.array(data_dict[urls[url]]['zarr_store']['acquisition_date_img2'][:], dtype='timedelta64[D]')
    
    # Get some arrays
    index_sort = np.argsort(np.datetime64('1970-01-01') + np.array(data_dict[urls[url]]['zarr_store']['mid_date'][:], dtype='timedelta64[D]'))
    mid_dates = mid_dates[index_sort]
    im1 = im1[index_sort]
    im2 = im2[index_sort]


    # Check which im is the smallest (first image, it changes depending on ITS_LIVE's version)
    if im2[0] < im1[0]:
        temp = im1
        im1 = im2
        im2 = temp


    # Create the date array with the new interval dates
    dates_nonum = np.arange(mid_dates[0], mid_dates[-1], timedelta(days=day_interval)).astype(np.datetime64)

    # Convert to numerical
    dates = (dates_nonum - np.datetime64('1970-01-01T00:00:00Z'))/np.timedelta64(1, 's')
    dt_start = (im1 - np.datetime64('1970-01-01T00:00:00Z'))/np.timedelta64(1, 's')
    dt_end = (im2 - np.datetime64('1970-01-01T00:00:00Z'))/np.timedelta64(1, 's')

    # --------------- DESIGN MATRICES --------------- 

    # Initialize matrix
    A_m = np.zeros((mid_dates.shape[0],dates.shape[0]))

    # We have to iterate through the satellite pairs that actually gave a measurement
    for j in range(1, len(mid_dates)):
    # current contents of your for loop

        # Find the middate that is the closest to dt_start (supequal)
        start = np.argmin(np.abs(dates-dt_start[j]))

        # Find the middate that is closest to dt_end (infequal)
        end = np.argmin(dt_end[j] - dates[dates <= dt_end[j]])

        # Divide 1 by the amount of middates between d_start and d_end 
        if end == A_m.shape[1]-1: # If the mid_date is at the end of the array (acquisition im2 equals last mid_date)
            A_m[j, start:] = 1/(1+A_m.shape[1]-start)
        else: # If the measurement is in A's bounds temporally (we can have a satellite pair with the 2nd pair being outside of our mid_dates)
            A_m[j, start:end+1] = 1/(1+end-start) # Attribute to each pixel in the timescale of the satellite pair, the 1/amount of pixel in the pairmid_dates.shape


    # Initialize regularization matrix
    if derivative == 1:
        reg_mat_Inv = np.zeros((A_m.shape[1] -1, A_m.shape[1]))

        for j in range(A_m.shape[1] -1):
            reg_mat_Inv[j, j] = -lamb/day_interval
            reg_mat_Inv[j, j+1] = lamb/day_interval

    elif derivative == 2:
        # Initialize 2nd derivative regularization matrix
        reg_mat_Inv = np.zeros((A_m.shape[1] -1, A_m.shape[1]))

        for j in range(A_m.shape[1] -2):
            reg_mat_Inv[j, j] = lamb/(day_interval**2)
            reg_mat_Inv[j, j+1] = -2*lamb/(day_interval**2)
            reg_mat_Inv[j, j+2] = lamb/(day_interval**2)
            
    data_dict[urls[url]]['A_m'] = A_m
    data_dict[urls[url]]['reg_mat_Inv'] = reg_mat_Inv
    data_dict[urls[url]]['mission'] = mission
    data_dict[urls[url]]['index_sort'] = index_sort
    data_dict[urls[url]]['inds_mission'] = inds_mission
    data_dict[urls[url]]['dates'] = dates_nonum
            
    return data_dict

In [ ]:
#| export 
def Inv_reg(vObs, data, fillvalue):
 
    # Grab observed velocities
    vObs = vObs[data['index_sort']]
    
    # Filter out the missions we don't want
    if mission:
        vObs = vObs[data['inds_mission']]  
    
    # Mask the NaNs so we don't compute the inversion for empty rows
    mask = np.logical_not(np.equal(vObs, fillvalue))
    
    # Create a masked velocity vector
    vObs_masked = vObs[mask]
    
    # Append regularization terms to dObs
    vObs_masked= np.hstack((vObs_masked, np.zeros((data['reg_mat_Inv'].shape[0]))))
     
    # Assemble the design matrix
    A_des = np.vstack((data['A_m'][mask], data['reg_mat_Inv']))
    
    # Invert the velocities
    vInv = np.linalg.solve(A_des.T@A_des,A_des.T@vObs_masked)
    
    return vInv

# **Select Your Parameters**

In [ ]:
mission = None # 'None' if you want all the data, 'S1' for Sentinel-1 only, 'L' for Landsat only, etc.. .
lamb = 10 # Smoothing coefficient: the higher the value, the more the inversion favors a smooth output. BAD for surging glaciers, GOOD for non-surging glaciers
derivative = 2 # Derivative degree for the inversion. Doesn't change much unless you have a specific reasong to choose 1 or 2 (1st or 2nd derivative)
day_interval = 12 # Amount of days in between each inversion value. The higher, the faster the inversion. But you also lose in temporal resolution. 12 here because Sentinel-1 repeat-time is 12.


## Get extents of each datacube

In [ ]:
gdf_list

[                       id   CENLON  ZMAX   BGNDATE  ZMIN           RGIID  \
 0  RGI_SouthAsiaWest.6794  76.4047  8569  20010721  3385  RGI60-14.06794   
 
    ASPECT   CENLAT  SLOPE  ZMED  ...  TERMTYPE  O2REGION  STATUS   ENDDATE  \
 0     296  35.7416   23.8  5393  ...         0         2       0  -9999999   
 
    FORM SURGING         GLIMSID O1REGION             NAME  \
 0     0       3  G076405E35742N       14  Baltoro Glacier   
 
                                             geometry  
 0  MULTIPOLYGON (((76.54920 35.92760, 76.54830 35...  
 
 [1 rows x 24 columns]]

In [ ]:
for url in tqdm(range(len(urls))):
    X_tot, Y_tot, X_valid, Y_valid = get_extents(urls[url], X_tot, Y_tot, X_valid, Y_valid, data_dict, mission, lamb, derivative, day_interval)

100%|██████████| 1/1 [00:28<00:00, 28.81s/it]


### For each datacube's valid indices (on the GOI), get the corresponding coordinates in a layer that gathers all the GOI parts spread accross the datacubes

In [ ]:
# Create Eastings and Northings arrays based on the Eastings and Northings of the datacubes
X_arr = np.unique(np.hstack(X_tot))
Y_arr = np.unique(np.hstack(Y_tot))

In [ ]:
# Crop to the GOI (so we avoid over-filling our matrix with NaNs)
x_min = np.where(np.min(np.hstack(X_valid)) == X_arr)[0][0]
x_max = np.where(np.max(np.hstack(X_valid)) == X_arr)[0][0]
y_min = np.where(np.min(np.hstack(Y_valid)) == Y_arr)[0][0]
y_max = np.where(np.max(np.hstack(Y_valid)) == Y_arr)[0][0]

In [ ]:
# And now search the indices corresponding to the coordinates 
x_matches = np.hstack([[np.where(i == X_arr[min(x_min-1, x_max+1):max(x_min-1, x_max+1)])[0][0] for i in row] for row in X_valid]).astype(int)
y_matches = np.hstack([[np.where(i == Y_arr[min(y_min-1, y_max+1):max(y_min-1, y_max+1)])[0][0] for i in row] for row in Y_valid]).astype(int)

IndexError: index 0 is out of bounds for axis 0 with size 0

In [ ]:
# Create an array representing the glacier
template = np.zeros((len(Y_arr[min(y_min-1, y_max+1):max(y_min-1, y_max+1)]), len( X_arr[min(x_min-1, x_max+1):max(x_min-1, x_max+1)])))
template[y_matches, x_matches] = 1

Verify that the glacier looks as we would expect:

In [ ]:
plt.pcolormesh(X_arr[x_min-1:x_max+1], Y_arr[y_min-1:y_max+1], template)

NameError: name 'template' is not defined

### Create the design matrices for each datacube

In [ ]:
for i in tqdm(range(len(urls))):
    design_matrices(i, mission, lamb, derivative, day_interval)

### Gather dates for each datacube

In [ ]:
# Get the total amount of temporal steps
ind_tot = []
for i in urls:
    ind_tot.append(data_dict[i]['dates'])

ind_tot = np.unique(np.hstack(ind_tot))

for i in urls:
    data_dict[i]['ind_tot'] = np.array([np.where(c == ind_tot)[0][0] for c in data_dict[urls[0]]['dates']])

### Calculate the point-inversion for all the GOI pixels in the datacubes for Vx and Vy.

In [ ]:
vxInv = np.zeros((len(ind_tot), template.shape[0], template.shape[1]))
vyInv = np.zeros((vxInv.shape))

# Define the total number of iterations
total_iterations = len(y_matches)

# Create a tqdm object with dynamic_ncols=False to suppress intermediate updates
# Create a tqdm object with a larger mininterval to suppress intermediate updates
progress_bar = tqdm(total=total_iterations, dynamic_ncols=False, mininterval=1.0)


i = 0
for c in range(len(urls)):
    valid_idx = data_dict[urls[c]]['valid_idx']
    fillvx = data_dict[urls[c]]['zarr_store']['vx'].fill_value
    fillvy = data_dict[urls[c]]['zarr_store']['vy'].fill_value
    
    for V in range(len(valid_idx[0])):
        vxObs = data_dict[urls[c]]['zarr_store']['vx'][:, valid_idx[0][V], valid_idx[1][V]]
        vyObs = data_dict[urls[c]]['zarr_store']['vy'][:, valid_idx[0][V], valid_idx[1][V]]
        vxInv[data_dict[urls[c]]['ind_tot'], y_matches[i], x_matches[i]] = Inv_reg(vxObs, data_dict[urls[c]], fillvx)
        vyInv[data_dict[urls[c]]['ind_tot'], y_matches[i], x_matches[i]] = Inv_reg(vyObs, data_dict[urls[c]], fillvy)
        
        i += 1
        progress_bar.update(1)  # Update the progress bar

# Close the progress bar
progress_bar.close()

### Save results to a netcdf file

In [ ]:
# Create a new dataset with vx and vy, using attributes from 'ds'
new_ds = xr.Dataset(
    {
        "vx": (["time", "y", "x"], vxInv),
        "vy": (["time", "y", "x"], vyInv),
    },
    coords={
        "time": ind_tot,
        "x": X_arr[x_min-1:x_max+1],
        "y": Y_arr[y_min-1:y_max+1],
    },
    attrs=data_dict[urls[0]]['zarr_store'].attrs,
).chunk({'time': 1, 'x': 100, 'y': 100})

from dask.diagnostics import ProgressBar
write_job = new_ds.to_netcdf('Inverted_Cube.nc', compute=False)
with ProgressBar():
    print(f"Writing to {'Inverted_Cube.nc'}")
    write_job.compute()

In [ ]:
V = 100
c = 0

plt.figure(figsize=(15, 10))
test = data_dict[urls[c]]['zarr_store']['vx'][:, valid_idx[0][V], valid_idx[1][V]].astype(np.float32)
test[test<-25000] = np.nan
plt.scatter(data_dict[urls[c]]['zarr_store']['mid_date'][:], test, label='Observed')
plt.plot(ind_tot, vxInv[:, y_matches[500], x_matches[500]], color = 'orange',label='Inverted')
plt.legend()